In [8]:
import numpy                 as np

import keras                        
from   keras.optimizers      import Adam
from   models.example_model  import *
from   sklearn.preprocessing import normalize
import sys
import argparse              as Ap

Using TensorFlow backend.


In [0]:
def dataPrepDecodaQuaternion(filename, isquat=True):

    nbTopics         = 250
    quaternionFactor = 4
    realChannel      = 3
    nbClasses        = 8
    raw              = open(filename, 'r').readlines()

    if(isquat):
        x            = np.ndarray(shape=(len(raw), nbTopics, quaternionFactor))
    else:
        x            = np.ndarray(shape=(len(raw), nbTopics, realChannel))
    y                = np.ndarray(shape=(len(raw), nbClasses))
    elementCpt       = 0
    documentCpt      = 0

    for doc in raw:
        elements   = doc.split('\t')[0].split(" ")
        nbElements = len(elements)

        # DATA
        for element in elements:
            components                                = element.split(',')

            if(isquat):
                x[documentCpt][elementCpt][0]   = components[0]
                x[documentCpt][elementCpt][1]   = components[1]
                x[documentCpt][elementCpt][2]   = components[2]
                x[documentCpt][elementCpt][3]   = components[3]
            else:
                x[documentCpt][elementCpt][0]   = components[1]
                x[documentCpt][elementCpt][1]   = components[2]
                x[documentCpt][elementCpt][2]   = components[3]

            elementCpt += 1
        elementCpt   = 0

        # LABELS
        labels   = doc.split('\t')[1].split(" ")
        labelCpt = 0
        for label in labels:
            values                   = label.split(',')
            y[documentCpt][labelCpt] = values[0]
            labelCpt                += 1
        labelCpt = 0
        documentCpt += 1
    
    return x,y





In [0]:
x_train, y_train = dataPrepDecodaQuaternion('decoda/250_TRAIN_Q.data',isquat=True)
x_dev,   y_dev   = dataPrepDecodaQuaternion('decoda/250_DEV_Q.data',  isquat=True)
x_test,  y_test  = dataPrepDecodaQuaternion('decoda/250_TEST_Q.data', isquat=True)

In [11]:
print('Train size : '+str(x_train.shape[0]))
print('Dev size   : '+str(x_dev.shape[0]))
print('Test size  : '+str(x_test.shape[0]))

print('Parameters    -------------------------------')
opt = Adam(lr = 0.0005)
print('learning rate   : '+str(0.0005))
print('Model type      : '+"QCNN")

Train size : 739
Dev size   : 174
Test size  : 325
Parameters    -------------------------------
learning rate   : 0.0005
Model type      : QCNN


In [0]:
from   complexnn                             import *
import keras
from   keras.layers                          import *
from   keras.models                          import Model
import keras.backend                         as     K
import numpy                                 as     np



input_seq = Input((250,4))
        

conv    = QuaternionConv1D(32, 3, strides=1, activation='relu', padding="same")(input_seq)
pool    = AveragePooling1D(2, padding='same')(conv)
conv2   = QuaternionConv1D(64, 3, strides=1, activation='relu', padding="same")(pool)
pool2   = AveragePooling1D(4, padding='same')(conv2)

        
flat    = Flatten()(pool2)
dense  = QuaternionDense(256, activation='relu')(flat)
output = Dense(8, activation='softmax')(dense)    
classifier=Model(input_seq, output)



In [15]:
print(' ')
print('Model Summary ----------------------------')
print(classifier.summary())


 
Model Summary ----------------------------
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 250, 4)            0         
_________________________________________________________________
quaternion_conv1d_1 (Quatern (None, 250, 128)          512       
_________________________________________________________________
average_pooling1d_1 (Average (None, 125, 128)          0         
_________________________________________________________________
quaternion_conv1d_2 (Quatern (None, 125, 256)          24832     
_________________________________________________________________
average_pooling1d_2 (Average (None, 32, 256)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
quaternion_den

In [16]:
classifier.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(x_train, y_train,
                validation_data=(x_dev,y_dev),
                epochs=15,
                batch_size=3)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 739 samples, validate on 174 samples
Epoch 1/15





739/739 [==============================] - 16s 21ms/step - loss: 1.0238 - acc: 0.6820 - val_loss: 0.6832 - val_acc: 0.7931
Epoch 2/15
739/739 [==============================] - 1s 2ms/step - loss: 0.5185 - acc: 0.8430 - val_loss: 0.6034 - val_acc: 0.7759
Epoch 3/15
739/739 [==============================] - 1s 2ms/step - loss: 0.4151 - acc: 0.8755 - val_loss: 0.6381 - val_acc: 0.7816
Epoch 4/15
739/739 [==============================] - 2s 2ms/step - loss: 0.3459 - acc: 0.8769 - val_loss: 0.5148 - val_acc: 0.8448
Epoch 5/15
739/739 [==============================] - 1s 2ms/step - loss: 0.2456 - acc: 0.9161 - val_loss: 0.5001 - val_acc: 0.8276
Epoch 6/15
739/739 [==============================] - 1s 2ms/step - loss: 0.1927 - acc: 0.9418 - val_loss: 0.4988 - val_acc: 0.8621
Epoch 7/15
739/739 [==============================] - 1s

In [17]:
loss, acc = classifier.evaluate(x_test,y_test)
print('Test Loss = '+str(loss)+' | Test accuracy = '+str(acc))
print("That's All Folks :p ")

325/325 [==============================] - 0s 389us/step
Test Loss = 0.831963334450355 | Test accuracy = 0.8123076923993917
That's All Folks :p 


In [0]:
x_train, y_train = dataPrepDecodaQuaternion('decoda/250_TRAIN_Q.data',isquat=False)
x_dev,   y_dev   = dataPrepDecodaQuaternion('decoda/250_DEV_Q.data',  isquat=False)
x_test,  y_test  = dataPrepDecodaQuaternion('decoda/250_TEST_Q.data', isquat=False)


In [0]:
input_seq = Input((250,3))

conv    = Conv1D(32, 3, strides=1, activation='relu', padding="same")(input_seq)
pool    = AveragePooling1D(2, padding='same')(conv)
conv2   = Conv1D(64, 3, strides=1, activation='relu', padding="same")(pool)
pool2   = AveragePooling1D(4, padding='same')(conv2)

       
flat    = Flatten()(pool2)
dense  = Dense(256, activation='relu')(flat)

output = Dense(8, activation='softmax')(dense)
classifier=Model(input_seq, output)

In [20]:
print('Train size : '+str(x_train.shape[0]))
print('Dev size   : '+str(x_dev.shape[0]))
print('Test size  : '+str(x_test.shape[0]))

print('Parameters    -------------------------------')
opt = Adam(lr = 0.0005)
print('learning rate   : '+str(0.0005))
print('Model type      : '+str("CNN"))

Train size : 739
Dev size   : 174
Test size  : 325
Parameters    -------------------------------
learning rate   : 0.0005
Model type      : CNN


In [21]:
print(' ')
print('Model Summary ----------------------------')
print(classifier.summary())


 
Model Summary ----------------------------
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 250, 3)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 250, 32)           320       
_________________________________________________________________
average_pooling1d_3 (Average (None, 125, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 125, 64)           6208      
_________________________________________________________________
average_pooling1d_4 (Average (None, 32, 64)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense

In [22]:
classifier.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(x_train, y_train,
                validation_data=(x_dev,y_dev),
                epochs=15,
                batch_size=3)


Train on 739 samples, validate on 174 samples
Epoch 1/15
739/739 [==============================] - 2s 2ms/step - loss: 1.2963 - acc: 0.5765 - val_loss: 0.7835 - val_acc: 0.7701
Epoch 2/15
739/739 [==============================] - 1s 1ms/step - loss: 0.6418 - acc: 0.8119 - val_loss: 0.6053 - val_acc: 0.8276
Epoch 3/15
739/739 [==============================] - 1s 1ms/step - loss: 0.5139 - acc: 0.8390 - val_loss: 0.5074 - val_acc: 0.8391
Epoch 4/15
739/739 [==============================] - 1s 1ms/step - loss: 0.4605 - acc: 0.8444 - val_loss: 0.6040 - val_acc: 0.7816
Epoch 5/15
739/739 [==============================] - 1s 1ms/step - loss: 0.4248 - acc: 0.8633 - val_loss: 0.5649 - val_acc: 0.8448
Epoch 6/15
739/739 [==============================] - 1s 1ms/step - loss: 0.3875 - acc: 0.8701 - val_loss: 0.4997 - val_acc: 0.8621
Epoch 7/15
739/739 [==============================] - 1s 2ms/step - loss: 0.3454 - acc: 0.8917 - val_loss: 0.5052 - val_acc: 0.8621
Epoch 8/15
739/739 [==========

In [23]:
loss, acc = classifier.evaluate(x_test,y_test)
print('Test Loss = '+str(loss)+' | Test accuracy = '+str(acc))
print("That's All Folks :p ")

325/325 [==============================] - 0s 153us/step
Test Loss = 0.5396377257200388 | Test accuracy = 0.8307692311360286
That's All Folks :p 
